In [0]:
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/industry_energy/energy_train.csv
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/industry_energy/energy_test.csv

In [0]:
import pandas as pd
df = pd.read_csv('energy_train.csv')
test_df = pd.read_csv('energy_test.csv')

data = df.fillna(df.mean())
test_data = test_df.fillna(test_df.mean())

df = data.copy()
test_df = test_data.copy()

In [24]:
deg_col = ['degree_C1','degree_C2','degree_C3','degree_C4','degree_C5','degree_C6','degree_C7','degree_C8','degree_C9','degree_Cout'] 
moi_col = ['moisture_1','moisture_2','moisture_3','moisture_4','moisture_5','moisture_6','moisture_7','moisture_8','moisture_9','moisture_out']
ot_col = ['Pressure','Wind','Clarity','dew_index','random_variable_1','random_variable_2','id']

col = df[deg_col]
df['m_deg'] = col.mean(axis=1)
col = df[moi_col]
df['m_moi'] = col.mean(axis=1)

col = test_df[deg_col]
test_df['m_deg'] = col.mean(axis=1)
col = test_df[moi_col]
test_df['m_moi'] = col.mean(axis=1)

df.drop(columns=deg_col+moi_col+ot_col, inplace=True)
test_df.drop(columns=deg_col+moi_col+ot_col, inplace=True)

df['date'] = pd.to_datetime(df['date'])
test_df['date'] = pd.to_datetime(test_df['date'])

df.columns = ['ds','y','lumino','m_deg']
test_df.columns = ['ds','lumino','m_deg']

ValueError: ignored

In [25]:
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation

train_prophet = df.copy()
test_prophet = test_df.copy()

prophet = Prophet()
prophet.add_regressor('lumino')
prophet.add_regressor('m_deg')
# prophet.add_regressor('m_moi')
prophet.fit(train_prophet)
forecast = prophet.predict(test_prophet)
_ = prophet.plot_components(forecast)

ValueError: ignored

In [0]:
y_pred = forecast['yhat']
y_pred = y_pred.round(0).astype('int64')
output = pd.DataFrame({'id': test_data.id,
                       'WattHour': y_pred})
output.to_csv('submission.csv', index=False)
from google.colab import files
files.download('submission.csv')